# Spikes per stim: Ganglion Cells in the Retina
- **Author:** David Felipe
- **Contact:** https://github.com/davidnfu0
- **Last Modification:** January 25, 2024
- **Description:** In this file, the spikes of each template will be grouped into the corresponding stimulus.
- **Note:** This notebook will take a considerable amount of time to execute, as the procedure is computationally complex.

## Introduction

### Importing libraries

In [1]:
import sys
import pickle
import h5py
import numpy as np
import pandas as pd
from collections import deque

In [2]:
sys.path.append('../../')

In [3]:
from scripts import load_yaml_config

### Paths, constants and configuration

In [4]:
configPath = "../../config/"
config = load_yaml_config(configPath + "general_config.yml")

NEW_EVENT_LIST_PATH = "../.." + config["paths"]["data"]["new_event_list"]
SPIKE_TIMES_PATH = "../.." + config["paths"]["data"]["result_merged"]

### Loading data

In [5]:
NEW_EVENT_LIST = pd.read_csv(NEW_EVENT_LIST_PATH)

units = h5py.File(SPIKE_TIMES_PATH, 'r')
spt_units = units["spiketimes"]
SPIKE_TIMES = {}
for elem in spt_units.keys():
    SPIKE_TIMES[elem] = np.array(spt_units[elem])
units.close()

In [6]:
toDel = []
for temp, spikes in SPIKE_TIMES.items():
    if len(spikes) <= 1000:
        toDel.append(temp)
for temp in toDel:
    del SPIKE_TIMES[temp]
TEMPS = list(SPIKE_TIMES.keys())

## Adding the data

In [7]:
stimDict = {}

protocol_descriptions = [f"{p}.{d}" for p, d in zip(NEW_EVENT_LIST["protocol_name"], NEW_EVENT_LIST["extra_description"])]

for temp in TEMPS:
    stimDict[temp] = {}
    for pro in protocol_descriptions:
        stimDict[temp][pro] = {}
    for pro, rep in zip(protocol_descriptions, NEW_EVENT_LIST["repetition_name"]):
        stimDict[temp][pro][rep] = deque()

for temp, spikes in zip(TEMPS, SPIKE_TIMES.values()):
    for startTime, endTime, protocol, rep, description in zip(NEW_EVENT_LIST["start_event"], NEW_EVENT_LIST["end_event"], NEW_EVENT_LIST["protocol_name"], NEW_EVENT_LIST["repetition_name"], NEW_EVENT_LIST["extra_description"]):
        protocolDescription = f"{protocol}.{description}"
        for spike in spikes:
            if startTime <= spike <= endTime:
                stimDict[temp][protocolDescription][rep].append(spike - startTime)

## Export data

In [8]:
with open("../../" + config["paths"]["data_cache"]["STIM_DICT"], 'wb') as output: 
    pickle.dump(stimDict, output)

___